In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
import numpy as np
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
#Montar o drive:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#arquivo = "/content/drive/MyDrive/Mestrado/Mestrado_Otacílio_Raphael/DataSetDGA/DataSet_DGA_(Rev1)T1.xlsx"
arquivo = "/content/drive/MyDrive/Mestrado/Mestrado_Otacílio_Raphael/DataSetDGA/DataSetDGA(Rev1)T1_S.Outiliers.xlsx"

In [ ]:
DataSet = pd.read_excel(arquivo)                                                             # Sintaxe para nomear DataSet.
DataSet.head()

,Unnamed: 0,H2,CH4,C2H4,C2H6,C2H2,NTE,NF,Rótulo
0,0,21.00,63.0,4.0,87.0,1.0,0,0,NO
1,1,0.01,61.0,6.0,81.0,1.0,0,0,NO
2,2,0.01,58.0,5.0,77.0,1.0,0,0,NO
3,3,0.01,56.0,6.0,73.0,1.0,0,0,NO
4,4,0.01,58.0,5.0,75.0,1.0,0,0,NO


In [ ]:
gases = ['H2', 'CH4', 'C2H4', 'C2H6', 'C2H2']
X = DataSet[gases].values

In [ ]:
classe = DataSet['NTE'].values

In [ ]:
classe_dummy = np_utils.to_categorical(classe)

In [ ]:
Xl = np.log(X)
#Xln = np.log(X)

In [ ]:
# Normalizar DataSet:
from sklearn.preprocessing import StandardScaler
previsores = StandardScaler().fit_transform(Xl)
print('Média antes:',np.mean(X))
print('Média depois:',np.mean(previsores))
print('Variância antes:',np.std(X))
print('Variância depois:',np.std(previsores))
np.isnan(previsores)

Média antes: 138.30720459081837
Média depois: -6.52394527843605e-17
Variância antes: 1617.3875441693656
Variância depois: 0.9999999999999999


array([[False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       ...,
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False]])

In [ ]:
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe_dummy, test_size=0.25, random_state=25)


Atribuição numérica aos dados:

Classe Normal = 0;

Falha Térmica = 1;

Falha elétrica = 2.

In [ ]:
from keras import initializers
classificador = Sequential()
classificador.add(Dense(units = 12, activation = 'selu', kernel_initializer = 'normal', input_dim = 5))
classificador.add(Dense(units = 12, activation = 'selu', kernel_initializer = 'normal'))
classificador.add(Dense(units = 8, activation = 'selu', kernel_initializer = 'normal'))
classificador.add(Dense(units = 6, activation = 'selu', kernel_initializer = 'normal'))
classificador.add(Dense(units = 3, activation = 'softmax'))
otimizador = keras.optimizers.Adam(lr=0.01, decay=0.0001, clipvalue=0.5)
classificador.compile(optimizer = otimizador, loss = 'categorical_crossentropy',
                      metrics = ['categorical_accuracy'])
classificador.fit(previsores_treinamento, classe_treinamento, batch_size = 10,
                  epochs = 100)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
151/151 [==============================] - 1s 2ms/step - loss: 0.5196 - categorical_accuracy: 0.7951
Epoch 2/100
151/151 [==============================] - 0s 2ms/step - loss: 0.2943 - categorical_accuracy: 0.8869
Epoch 3/100
151/151 [==============================] - 0s 2ms/step - loss: 0.2745 - categorical_accuracy: 0.8989
Epoch 4/100
151/151 [==============================] - 0s 2ms/step - loss: 0.2667 - categorical_accuracy: 0.9029
Epoch 5/100
151/151 [==============================] - 0s 2ms/step - loss: 0.2631 - categorical_accuracy: 0.9035
Epoch 6/100
151/151 [==============================] - 0s 3ms/step - loss: 0.2437 - categorical_accuracy: 0.9175
Epoch 7/100
151/151 [==============================] - 0s 3ms/step - loss: 0.2425 - categorical_accuracy: 0.9022
Epoch 8/100
151/151 [==============================] - 0s 3ms/step - loss: 0.2333 - categorical_accuracy: 0.9155
Epoch 9/100
151/151 [==============================] - 0s 2ms/step - loss: 0.2427 - categorical_

In [ ]:
resultado = classificador.evaluate(previsores_teste, classe_teste)



16/16 [==============================] - 0s 2ms/step - loss: 0.1944 - categorical_accuracy: 0.9481


In [ ]:
previsoes = classificador.predict(previsores_teste)
previsoes = (previsoes > 0.5)

16/16 [==============================] - 0s 1ms/step


In [ ]:
import numpy as np
classe_teste2 = [np.argmax(t) for t in classe_teste]
previsoes2 = [np.argmax(t) for t in previsoes]

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(previsoes2, classe_teste2)

In [ ]:
matriz

array([[315,   6,   9],
       [  6,  98,   3],
       [  1,   1,  62]])

## GridSearch com Validação Cruzada

In [ ]:
def criarRede(optimizer, loos, kernel_initializer, activation, neurons):
  classificador = Sequential()
  classificador.add(Dense(units = neurons, activation = activation, kernel_initializer = kernel_initializer, input_dim = 5))
  classificador.add(Dropout(0.2))
  classificador.add(Dense(units = 12, activation = activation, kernel_initializer = kernel_initializer))
  classificador.add(Dropout(0.2))
  classificador.add(Dense(units = neurons, activation = activation, kernel_initializer = kernel_initializer))
  classificador.add(Dropout(0.2))
  classificador.add(Dense(units = 6, activation = activation, kernel_initializer = kernel_initializer))
  classificador.add(Dropout(0.2))
  classificador.add(Dense(units = 3, activation = 'softmax'))
  #otimizador = keras.optimizers.Adam(lr=0.01, decay=0.0001, clipvalue=0.5)
  classificador.compile(optimizer = optimizer, loss = loos,
                      metrics = ['categorical_accuracy'])
  return classificador

In [ ]:
classificador = KerasClassifier(build_fn = criarRede)
parametros = {'batch_size': [10],
              'epochs': [50, 100],
              'optimizer': ['adam'],
              'loos': ['categorical_crossentropy'],
              'kernel_initializer': ['random_uniform', 'normal'],
              'activation': ['relu', 'tanh', 'selu'],
              'neurons': [12, 8]}
grid_search = GridSearchCV(estimator = classificador,
                           param_grid = parametros,
                           scoring = 'accuracy',
                           cv = 5)


<ipython-input-17-8d4631ae7098>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classificador = KerasClassifier(build_fn = criarRede)


In [ ]:
grid_search = grid_search.fit(previsores, classe)



In [ ]:
melhores_parametros = grid_search.best_params_

In [ ]:
melhores_parametros

{'activation': 'selu',
 'batch_size': 10,
 'epochs': 100,
 'kernel_initializer': 'normal',
 'loos': 'categorical_crossentropy',
 'neurons': 12,
 'optimizer': 'adam'}

In [ ]:
melhor_precisao = grid_search.best_score_

In [ ]:
melhor_precisao

0.7384999999999999

In [ ]:
#classificador = KerasClassifier(build_fn = criarRede, epochs = 100, batch_size = 10)

In [ ]:
#resultados = cross_val_score(estimator = classificador, X = previsores, y = classe, cv = 10, scoring = 'accuracy')

In [ ]:
#media = resultados.mean()
#media

In [ ]:
# quanto maior o desvio - maior será o overfintting
#desvio = resultados.std()
#desvio